In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_276360/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_6_5,0.694230,0.002881,0.197950,0.617281,0.041145,0.591933,0.561853,0.064290,0.072207,0.202842,0.280167,0.204645,136.381304,314.234203,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_10_1,0.680588,-0.880322,-0.056065,0.756842,0.042981,1.116241,0.739795,0.044068,0.078083,0.207318,0.248052,0.209161,136.294008,314.146907,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_10_2,0.680466,-1.013094,-0.067138,0.709365,0.042997,1.195059,0.747552,0.052673,0.073778,0.207358,0.247763,0.209201,136.293241,314.146140,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_6_6,0.678526,-0.047093,0.181796,0.417083,0.043258,0.621600,0.573169,0.097920,0.071685,0.207986,0.243196,0.209835,136.281135,314.134034,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_6_4,0.676032,0.001772,0.186674,0.669763,0.043594,0.592591,0.569752,0.055474,0.075982,0.208791,0.237326,0.210647,136.265682,314.118582,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,model_1_7_8,-1.356314,-0.290404,0.096591,-9.277585,0.317071,0.766040,0.632857,1.455483,0.148495,0.563090,-4.547156,0.568096,132.297262,310.150161,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
86,model_1_10_8,-1.372632,-2.329234,-0.136147,-7.132941,0.319266,1.976377,0.795894,1.473958,0.133895,0.565037,-4.585572,0.570059,132.283459,310.136358,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
87,model_1_7_7,-1.385748,-0.272981,0.107658,-9.395061,0.321031,0.755697,0.625104,1.472119,0.149300,0.566596,-4.616449,0.571633,132.272433,310.125332,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
88,model_1_7_6,-1.422913,-0.264518,0.117689,-9.554988,0.326032,0.750673,0.618077,1.494768,0.150481,0.570992,-4.703942,0.576068,132.241518,310.094417,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
